In [401]:
from rpy2 import robjects
from itertools import combinations

import argparse
import numpy as np
import pandas as pd
import time
import os

In [347]:
data = pd.read_csv('./test/t180-nuclei.csv', sep=',', header=None)

In [250]:
#compute distance
number = data.shape[0]
data_values = data.iloc[:,:3]

df = pd.DataFrame(columns = ['cellA', 'cellB', 'distance'])
for i in range(number):
    for j in range(i+1, number):
        dis = np.linalg.norm(data_values.iloc[i,:] - data_values.iloc[j,:])
        df_row = pd.DataFrame([[data.iloc[i,3], data.iloc[j,3], dis]],
                              columns = ['cellA', 'cellB', 'distance'])
        df = df.append(df_row)
        

In [235]:
#compute distance rate
df_rate = pd.DataFrame(columns=['cellA', 'cellB', 'distance_rate'])
number = data.shape[0]
data_values = data.iloc[:,:3]
for i in range(number):
    df_rate_ij = pd.DataFrame(columns=['cellA', 'cellB', 'distance_rate'])
    for j in range(number):
        dis = np.linalg.norm(data_values.iloc[i,:] - data_values.iloc[j,:])
        df_rate_row = pd.DataFrame([[data.iloc[i,3], data.iloc[j,3], dis]],
                          columns = ['cellA', 'cellB', 'distance_rate'])
        df_rate_ij = df_rate_ij.append(df_rate_row)   
    dis_ij = df_rate_ij['distance_rate']    
    df_rate_ij['distance_rate'] =  dis_ij / dis_ij.max()     
    df_rate = df_rate.append(df_rate_ij)    
#remove 0 distance rate
df_rate = df_rate[df_rate.iloc[:,2] != 0]

In [ ]:
#########################################################

In [406]:
picture_num = 5
for n in range(picture_num):
    data = pd.read_csv('./test/t%03d-nuclei.csv' % (n+1), sep=',', header=None)
    print(data)

     0    1     2   3
0  291  167  17.0  AB
1  359  257  17.0  P1
     0    1     2    3
0  280  193  15.0  ABp
1  284  160  16.0  ABa
2  371  238  16.0   P1
     0    1     2    3
0  278  196  16.0  ABp
1  286  157  15.0  ABa
2  370  239  16.0   P1
     0    1     2    3
0  290  205  15.0  ABp
1  286  160  14.0  ABa
2  365  218  16.0  EMS
3  378  254  16.0   P2
     0    1     2    3
0  293  210  14.0  ABp
1  278  158  15.0  ABa
2  363  211  16.0  EMS
3  383  253  17.0   P2


In [445]:
picture_num = 5
count_dict = {}
for n in range(picture_num):
    data = pd.read_csv('./test/t%03d-nuclei.csv' % (n+1), sep=',', header=None)
    cell_combination = list(combinations(data.iloc[:,3],2))
    for comb in cell_combination:
        if comb in count_dict:
            count_dict[comb] = count_dict[comb]+1
        else:
            count_dict[comb] = 1
pair = np.array(list(count_dict.keys()))
count = np.array(list(count_dict.values()))
stack_array = np.vstack([pair.transpose(), count]).transpose()
df_cotimes = pd.DataFrame(stack_array, columns=['cellA', 'cellB', 'coexistance_time'])

In [450]:
df_cotimes

,cellA,cellB,coexistance_time
0,AB,P1,1
1,ABp,ABa,4
2,ABp,P1,2
3,ABa,P1,2
4,ABp,EMS,2
5,ABp,P2,2
6,ABa,EMS,2
7,ABa,P2,2
8,EMS,P2,2


In [447]:
df_rate = df_cotimes.copy()

In [448]:
df_rate.iloc[:,2] = pd.to_numeric(df_rate.iloc[:,2]) / 180

In [451]:
df_rate

,cellA,cellB,coexistance_time
0,AB,P1,0.005556
1,ABp,ABa,0.022222
2,ABp,P1,0.011111
3,ABa,P1,0.011111
4,ABp,EMS,0.011111
5,ABp,P2,0.011111
6,ABa,EMS,0.011111
7,ABa,P2,0.011111
8,EMS,P2,0.011111


In [ ]:
df_rate.i